In [9]:
import torch
import numpy as np
import torch.nn.functional as torchF
import torch.nn as torchNN
import matplotlib.pyplot as plt
from IPython import display
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
%matplotlib inline

In [10]:
trainingData = datasets.CIFAR10(
    root = "../Database",
    train = True,
    download = True,
    transform = ToTensor(),
)

Files already downloaded and verified


In [11]:
BATCHSIZE = 64
trainData = DataLoader(trainingData, batch_size = BATCHSIZE)

In [12]:
class Network(torchNN.Module):
    def __init__(self):
        super(Network, self).__init__()

        # self.conv1 = torchNN.Conv2d(
        #     in_channels = 3,
        #     out_channels = 6,
        #     kernel_size = 5,
        #     stride = 1,
        #     padding = 2
        # )
        
        # self.pool = torchNN.MaxPool2d(
        #     kernel_size = 2, 
        #     stride = 2
        # )

        # self.conv2 = torchNN.Conv2d(
        #     in_channels = 6,
        #     out_channels = 16,
        #     kernel_size = 5,
        #     stride = 1,
        #     padding = 2
        # )

        # self.liner1 = torchNN.Linear(16 * 5 * 5, 128)
        # self.liner2 = torchNN.Linear(120, 32)
        # self.liner3 = torchNN.Linear(32, 10)

        self.conv1 = torchNN.Sequential(
            torchNN.Conv2d(
                in_channels = 3,
                out_channels = 16,
                kernel_size = 5,
                stride = 1,
                padding = 2
            ),
            # torchF.leaky_relu(),
            torchNN.LeakyReLU(),
            torchNN.MaxPool2d(2)
        )
        

        self.conv2 = torchNN.Sequential(
            torchNN.Conv2d(
                in_channels = 16,
                out_channels = 32,
                kernel_size = 5,
                stride = 1,
                padding = 2
            ),
            # torchF.leaky_relu(),
            torchNN.LeakyReLU(),
            torchNN.MaxPool2d(2)
        )

        self.liner = torchNN.Sequential(
            torchNN.Linear(32 * 8 * 8, 64),
            torchNN.LeakyReLU(),
            torchNN.Linear(64, 32),
            torchNN.LeakyReLU(),
            torchNN.Linear(32, 10)
        )

    
    def forward(self, inputData):
        x = self.conv1(inputData)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.liner(x)
        return x
    


In [13]:
model = Network()

lossFunction = torchNN.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)


In [18]:
history = {
    'loss':[]
}
for epoch in range(2):
    runtimeLoss = 0.0

    for i, data in enumerate(trainData):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = lossFunction(outputs, labels)
        loss.backward()
        optimizer.step()

        runtimeLoss += loss.item()
    print("EPOCH %d, loss = %.3f" % (epoch, runtimeLoss / BATCHSIZE))
    history['loss'].append(runtimeLoss / BATCHSIZE)
print(history['loss'])

{'loss': [19.200068905949593, 18.13692815043032]}
